PART 1 ASSIGNMENT

In [1]:

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import wget #to import csv file that has latitude and longitude of a location
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [2]:

#getting html text from the Wikipedia page url given in assignment
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
url[0:500]

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"X@HMugpAMNcAA4h6GIoAAA'

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(url,'lxml')
print(soup.prettify()[0:500])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"X@HM


In [4]:
zipcode_table_from_text = soup.find('table',{'class':'wikitable sortable'})

zipcode_table_from_text.text[0:500]

"\n\nPostal Code\n\nBorough\n\nNeighbourhood\n\n\nM1A\n\nNot assigned\n\nNot assigned\n\n\nM2A\n\nNot assigned\n\nNot assigned\n\n\nM3A\n\nNorth York\n\nParkwoods\n\n\nM4A\n\nNorth York\n\nVictoria Village\n\n\nM5A\n\nDowntown Toronto\n\nRegent Park, Harbourfront\n\n\nM6A\n\nNorth York\n\nLawrence Manor, Lawrence Heights\n\n\nM7A\n\nDowntown Toronto\n\nQueen's Park, Ontario Provincial Government\n\n\nM8A\n\nNot assigned\n\nNot assigned\n\n\nM9A\n\nEtobicoke\n\nIslington Avenue, Humber Valley Village\n\n\nM1B\n\nScarborough\n\nMalvern, Rouge\n\n\nM2B\n\nNot assigned\n\nNot assig"

In [5]:
table_data = zipcode_table_from_text.findAll('tr')

table_data[0:2]

[<tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>,
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>]

In [6]:

# extract the column names from first row
column_names = [th.text.replace("\n", "") for th in table_data[0].findAll('th')]
data_table = []

column_names

['Postal Code', 'Borough', 'Neighbourhood']

In [7]:

for tr in table_data:
    tmp = [td.text.replace("\n","") for td in tr.findAll('td')]
    if(len(tmp) > 0): data_table.append(tmp)

zip_df = pd.DataFrame(data_table, columns=column_names)
print('zip_df DataFrame size:', zip_df.shape)
zip_df

zip_df DataFrame size: (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [8]:
zip_df = zip_df[zip_df['Neighbourhood'] != 'Not assigned'].reset_index(drop=True)
zip_df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:

print('zip_df DataFrame size:', zip_df.shape)

zip_df DataFrame size: (103, 3)


PART 2 ASSIGNMENT


In [10]:

#Download the csv file
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [11]:
#read the csv file into DataFrame
lat_long_df = pd.read_csv('Geospatial_Coordinates.csv')
lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#create new DataFrame with co-ordinates data and neighborhoods dataframe by merging them on postal code column
city_df = pd.merge(zip_df, lat_long_df, on = "Postal Code", how = "inner")
city_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [13]:
#correcting Neighbourhood column name
city_df.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True) 
city_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
city_df.shape

(103, 5)

PART 3 ASSIGNMENT

In [15]:

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(city_df['Borough'].unique()),
        city_df.shape[0]
    )
)
print('The different buroughs are',city_df['Borough'].unique())

The dataframe has 10 boroughs and 103 neighborhoods.
The different buroughs are ['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga']


In [16]:
column_names = city_df.columns
data_table=[]
for index,row in city_df.iterrows():
    if("," in row['Neighborhood']):
        for nh in row['Neighborhood'].split(','):
            data_table.append([row["Postal Code"],row["Borough"], nh.strip(), row["Latitude"],row["Longitude"]])
    else:
        data_table.append([row["Postal Code"],row["Borough"], row['Neighborhood'], row["Latitude"],row["Longitude"]])
city_df = pd.DataFrame(data_table, columns = column_names)
city_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
...,...,...,...,...,...
212,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
213,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
214,M8Z,Etobicoke,South of Bloor,43.628841,-79.520999
215,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999


In [17]:
toronto_df = city_df[city_df.Borough.isin(['Downtown Toronto' , 'East Toronto', 'West Toronto' ,'Central Toronto' ])]
print('The size of DataFrame is ',toronto_df.shape)
toronto_df.head()

The size of DataFrame is  (78, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
6,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
7,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494
15,M5B,Downtown Toronto,Garden District,43.657162,-79.378937


In [18]:

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [19]:
# create map of Toronto burough using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [20]:

# @hidden_cell
CLIENT_ID = 'RC4M2LPPKTDZKVYREEK3BT1M4SX02PLPYGTC4O0AAZWNXACB' # your Foursquare ID
CLIENT_SECRET = 'BTGCGLUVIW1DEO3IYLSJX2SZQ2UGUKIYVCUUMUR0JAGSHVVD' # your Foursquare Secret
#ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [21]:
# type your answer here
LIMIT = 100
RADIUS = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude,
    VERSION, 
    20, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=RC4M2LPPKTDZKVYREEK3BT1M4SX02PLPYGTC4O0AAZWNXACB&client_secret=BTGCGLUVIW1DEO3IYLSJX2SZQ2UGUKIYVCUUMUR0JAGSHVVD&ll=43.6534817,-79.3839347&v=20180605&radius=20&limit=100'

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #print(url)    
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
#dataframe to store venue data for each neighborhood, also shows list fo neighborhoods explored by the function call
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park
Harbourfront
Queen's Park
Ontario Provincial Government
Garden District
Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
Adelaide
King
Dufferin
Dovercourt Village
Harbourfront East
Union Station
Toronto Islands
Little Portugal
Trinity
The Danforth West
Riverdale
Toronto Dominion Centre
Design Exchange
Brockton
Parkdale Village
Exhibition Place
India Bazaar
The Beaches West
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
Forest Hill Road Park
High Park
The Junction South
North Toronto West
Lawrence Park
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
University of Toronto
Harbord
Runnymede
Swansea
Moore Park
Summerhill East
Kensington Market
Chinatown
Grange Park
Summerhill West
Rathnelly
South Hill
Forest Hill SE
Deer Park
CN Tower
King and Spadina
Railway Lands
Harbourfront West
Bathurst Quay
South Niagara
Island airport
Rosedale
Stn A PO Boxes
St. James Tow

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(3174, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [25]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,97,97,97,97,97,97
Bathurst Quay,16,16,16,16,16,16
Berczy Park,58,58,58,58,58,58
Brockton,22,22,22,22,22,22
Business reply mail Processing Centre,17,17,17,17,17,17
...,...,...,...,...,...,...
Underground city,100,100,100,100,100,100
Union Station,100,100,100,100,100,100
University of Toronto,34,34,34,34,34,34


In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
print('The size of the toronto_onehot dataframe is ',toronto_onehot.shape)

The size of the toronto_onehot dataframe is  (3174, 235)


In [29]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Adelaide,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020619,...,0.0,0.0,0.0,0.0,0.00,0.010309,0.000000,0.0,0.00,0.0
1,Bathurst Quay,0.000000,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0
2,Berczy Park,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.017241,0.000000,0.0,0.00,0.0
3,Brockton,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0
4,Business reply mail Processing Centre,0.058824,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Underground city,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,...,0.0,0.0,0.0,0.0,0.01,0.010000,0.000000,0.0,0.01,0.0
72,Union Station,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.01,0.010000,0.000000,0.0,0.01,0.0
73,University of Toronto,0.029412,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.029412,0.0,0.00,0.0
74,Victoria Hotel,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,...,0.0,0.0,0.0,0.0,0.00,0.020000,0.000000,0.0,0.01,0.0


In [30]:
print('the size of toronto_grouped is ',toronto_grouped.shape)

the size of toronto_grouped is  (76, 235)


In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2   Restaurant  0.04
3        Hotel  0.04
4          Gym  0.03


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4           Airport  0.06


----Berczy Park----
            venue  freq
0     Coffee Shop  0.10
1    Cocktail Bar  0.05
2        Beer Bar  0.03
3  Farmers Market  0.03
4     Cheese Shop  0.03


----Brockton----
                    venue  freq
0                    Café  0.14
1          Breakfast Spot  0.09
2             Coffee Shop  0.09
3           Burrito Place  0.05
4  Furniture / Home Store  0.05


----Business reply mail Processing Centre----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3          Smoke Shop  0.06
4          Skate Park  0.06


----CN Tower----
              venue  freq
0    Airport Lounge  0.12
1   Airport S

         venue  freq
0  Coffee Shop  0.09
1         Café  0.07
2        Hotel  0.05
3   Restaurant  0.04
4          Gym  0.04


----Union Station----
                venue  freq
0         Coffee Shop  0.13
1            Aquarium  0.05
2                Café  0.04
3               Hotel  0.04
4  Italian Restaurant  0.03


----University of Toronto----
                 venue  freq
0                 Café  0.18
1               Bakery  0.06
2   Italian Restaurant  0.06
3  Japanese Restaurant  0.06
4            Bookstore  0.06


----Victoria Hotel----
                 venue  freq
0          Coffee Shop  0.13
1           Restaurant  0.07
2                 Café  0.06
3                Hotel  0.05
4  American Restaurant  0.04


----Yorkville----
            venue  freq
0            Café  0.16
1  Sandwich Place  0.16
2     Coffee Shop  0.11
3     Pizza Place  0.05
4    Burger Joint  0.05




In [32]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Thai Restaurant,Gym,Clothing Store,Sushi Restaurant,Bakery
1,Bathurst Quay,Airport Terminal,Airport Lounge,Airport Service,Boutique,Coffee Shop,Plane,Boat or Ferry,Harbor / Marina,Rental Car Location,Airport Gate
2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Cheese Shop,Farmers Market,Restaurant,Beer Bar,Basketball Stadium,Beach
3,Brockton,Café,Breakfast Spot,Coffee Shop,Furniture / Home Store,Burrito Place,Italian Restaurant,Stadium,Restaurant,Intersection,Bar
4,Business reply mail Processing Centre,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Comic Shop,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop


CLUST NEIGHBOURHOODS

In [34]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

In [35]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32)

In [36]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [37]:
toronto_merged = toronto_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Café,Breakfast Spot,Theater,Electronics Store,Spa,Beer Store
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Café,Breakfast Spot,Theater,Electronics Store,Spa,Beer Store
6,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Gym,Discount Store,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
7,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Gym,Discount Store,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
15,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Hotel,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Italian Restaurant,Fast Food Restaurant


In [38]:
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
167,Downtown Toronto,0,Airport Terminal,Airport Lounge,Airport Service,Boutique,Coffee Shop,Plane,Boat or Ferry,Harbor / Marina,Rental Car Location,Airport Gate
168,Downtown Toronto,0,Airport Terminal,Airport Lounge,Airport Service,Boutique,Coffee Shop,Plane,Boat or Ferry,Harbor / Marina,Rental Car Location,Airport Gate
169,Downtown Toronto,0,Airport Terminal,Airport Lounge,Airport Service,Boutique,Coffee Shop,Plane,Boat or Ferry,Harbor / Marina,Rental Car Location,Airport Gate
170,Downtown Toronto,0,Airport Terminal,Airport Lounge,Airport Service,Boutique,Coffee Shop,Plane,Boat or Ferry,Harbor / Marina,Rental Car Location,Airport Gate
171,Downtown Toronto,0,Airport Terminal,Airport Lounge,Airport Service,Boutique,Coffee Shop,Plane,Boat or Ferry,Harbor / Marina,Rental Car Location,Airport Gate
172,Downtown Toronto,0,Airport Terminal,Airport Lounge,Airport Service,Boutique,Coffee Shop,Plane,Boat or Ferry,Harbor / Marina,Rental Car Location,Airport Gate
173,Downtown Toronto,0,Airport Terminal,Airport Lounge,Airport Service,Boutique,Coffee Shop,Plane,Boat or Ferry,Harbor / Marina,Rental Car Location,Airport Gate


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Bakery,Park,Pub,Café,Breakfast Spot,Theater,Electronics Store,Spa,Beer Store
3,Downtown Toronto,1,Coffee Shop,Bakery,Park,Pub,Café,Breakfast Spot,Theater,Electronics Store,Spa,Beer Store
6,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Gym,Discount Store,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
7,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Gym,Discount Store,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
15,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Hotel,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Italian Restaurant,Fast Food Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
196,Downtown Toronto,1,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Seafood Restaurant,Steakhouse,Salad Place,Deli / Bodega
197,Downtown Toronto,1,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Seafood Restaurant,Steakhouse,Salad Place,Deli / Bodega
201,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Mediterranean Restaurant,Café,Pub,Hotel
202,East Toronto,1,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Comic Shop,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
123,Central Toronto,2,Jewelry Store,Park,Sushi Restaurant,Bus Line,Trail,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Wine Shop
124,Central Toronto,2,Jewelry Store,Park,Sushi Restaurant,Bus Line,Trail,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Wine Shop


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
162,Central Toronto,3,Coffee Shop,Light Rail Station,American Restaurant,Sushi Restaurant,Bank,Pub,Fried Chicken Joint,Restaurant,Bagel Shop,Supermarket
163,Central Toronto,3,Coffee Shop,Light Rail Station,American Restaurant,Sushi Restaurant,Bank,Pub,Fried Chicken Joint,Restaurant,Bagel Shop,Supermarket
164,Central Toronto,3,Coffee Shop,Light Rail Station,American Restaurant,Sushi Restaurant,Bank,Pub,Fried Chicken Joint,Restaurant,Bagel Shop,Supermarket
165,Central Toronto,3,Coffee Shop,Light Rail Station,American Restaurant,Sushi Restaurant,Bank,Pub,Fried Chicken Joint,Restaurant,Bagel Shop,Supermarket
166,Central Toronto,3,Coffee Shop,Light Rail Station,American Restaurant,Sushi Restaurant,Bank,Pub,Fried Chicken Joint,Restaurant,Bagel Shop,Supermarket


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
153,Central Toronto,4,Restaurant,Park,Trail,Tennis Court,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Wine Shop
154,Central Toronto,4,Restaurant,Park,Trail,Tennis Court,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Wine Shop
187,Downtown Toronto,4,Park,Playground,Trail,Wine Shop,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
